In [1]:
import torch 
from torch import nn
import numpy as np 
from tqdm import tqdm
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from torch.nn import functional as F

In [2]:
def constrained_search(
    n_subjects: int = 1024, 
    p_predicates: int = 1024,
    m_objects_per_predicate: int = 32,
    d_model: int = 16, 
):
    objects = []
    predicates = []
    for _ in range(p_predicates):
        mapping = torch.arange(0, m_objects_per_predicate).repeat(n_subjects // m_objects_per_predicate)
        mapping = mapping[torch.randperm(n_subjects)]
        embs = nn.init.normal_(torch.zeros(m_objects_per_predicate, d_model // 2))
        predicates.append(mapping)
        objects.append(embs[mapping])
    
    # y = nn.init.normal_(torch.zeros(n_subjects, d_model)).to(device)
    xs, ys = [], []
    for i in range(p_predicates):
        for j in range(i + 1, p_predicates):
            for k in range(m_objects_per_predicate):
                for l in range(m_objects_per_predicate):
                    xs.append(torch.concat([objects[i][k], objects[j][l]], dim=-1))
                    ys.append(int(((predicates[i] == k) & (predicates[j] == l)).any()))
    x = torch.stack(xs, dim=0)
    y = torch.tensor(ys)

    return x, y
x, y = constrained_search(
    n_subjects=32,
    p_predicates=32,
    m_objects_per_predicate=8,
    d_model=12,
)

In [3]:
x

tensor([[ 0.7070,  1.4108, -0.3484,  ...,  2.1461, -2.1936, -0.1641],
        [ 0.7070,  1.4108, -0.3484,  ..., -0.8891, -0.0532,  0.9548],
        [ 0.7070,  1.4108, -0.3484,  ..., -0.1183, -0.0856,  0.0340],
        ...,
        [-0.1364, -1.5150, -1.3489,  ...,  1.0739,  0.7339, -2.0477],
        [-0.1364, -1.5150, -1.3489,  ...,  0.7796, -0.2232, -1.7187],
        [-0.1364, -1.5150, -1.3489,  ..., -0.0434,  2.0079,  2.5560]])